# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver
    - chromedriver para o uso no google Chrome
    - geckodriver para o uso no firefox 

In [185]:
!pip install selenium

In [186]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#### Vamos criar um navegador virtual utilizado o programa Google Chrome onde a biblioteca "selenium" faz toda a automação

In [187]:
navegador = webdriver.Chrome() 

#### Passo 1: Pegar a cotação do dolar 

In [188]:
navegador.get("https://www.google.com/")
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)


- Vamos pegar o valor do dolar

In [189]:
cotacao_dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#### Passo 2: Pegar a cotação do euro

In [190]:
navegador.get("https://www.google.com/")
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)


- Vamos pegar o valor do euro

In [191]:
cotacao_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#### Passo 3: Pegar a cotação do ouro

In [192]:
navegador.get("https://www.melhorcambio.com/ouro-hoje")

- Vamos pegar o valor do ouro

In [193]:
cotacao_ouro = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",",".")


In [194]:
navegador.quit()

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [195]:
df_inicial = pd.read_excel("Produtos.xlsx")

In [196]:
df_inicial

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [197]:
df_inicial.loc[df_inicial["Moeda"]=="Dólar","Cotação"] = float(cotacao_dolar)
df_inicial.loc[df_inicial["Moeda"]=="Euro","Cotação"] = float(cotacao_euro)
df_inicial.loc[df_inicial["Moeda"]=="Ouro","Cotação"] = float(cotacao_ouro)

- Vamos atualizar o preço de compra

In [198]:
df_inicial["Preço de Compra"] = df_inicial["Cotação"]*df_inicial["Preço Original"]

- Vamos atualizar o preço de vendas

In [199]:
df_inicial["Preço de Venda"] = df_inicial["Preço de Compra"]*df_inicial["Margem"]

- Vamos mostrar a tabela atualizada

In [200]:
df_inicial

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.186800,5186.748132,1.40,7261.447385
1,Carro Renault,4500.00,Euro,5.598373,25192.676610,2.00,50385.353220
2,Notebook Dell,899.99,Dólar,5.186800,4668.068132,1.70,7935.715824
3,IPhone,799.00,Dólar,5.186800,4144.253200,1.70,7045.230440
4,Carro Fiat,3000.00,Euro,5.598373,16795.117740,1.90,31910.723706
5,Celular Xiaomi,480.48,Dólar,5.186800,2492.153664,2.00,4984.307328
6,Joia 20g,20.00,Ouro,317.870000,6357.400000,1.15,7311.010000


### Agora vamos exportar a nova base de preços atualizada

In [202]:
df_inicial.to_excel("produtos atualizado.xlsx", index=False)